In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from matplotlib import pyplot as plt
from glob import glob
import numpy as np
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.backend.tensorflow_backend import set_session
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from skimage.util import random_noise, crop
from sklearn.utils import shuffle
from skimage.transform import resize, rotate
%matplotlib inline

/home/rickya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [3]:
PATH = '00_input/'
IMSIZE = 100
MAXN = 6000
flip_indices = [
                (0, 6), (1, 7), (2, 4), (3, 5),
                (10, 16), (11, 17), (12, 14), (13, 15),
                (8, 18), (9, 19), (22, 26), (23, 27)
                ]

In [4]:
class Generator(object):
    import numpy as np
    def __init__(self,
                 X, 
                 y,
                 bs,
                 flip_ratio,
                 rotate_ratio,
                 noise_ratio,
                 zoom_ratio,
                 zoom_range,
                 flip_indices):
        self.X = X
        self.y = y
        self.bs = bs
        self.flip_ratio = flip_ratio
        self.rotate_ratio = rotate_ratio
        self.noise_ratio = noise_ratio
        self.zoom_ratio = zoom_ratio
        self.zoom_range = zoom_range
        
        self.size = X.shape[0]
        self.flip_indices = flip_indices
    
    def _random_indices(self, ratio):
        size = int(self.bs * ratio)
        return np.random.choice(self.bs, size, replace=False)
    
    def flip(self):
        indices = self._random_indices(self.flip_ratio)
        self.inputs[indices] = self.inputs[indices, :, ::-1]
        self.targets[indices, ::2] = self.targets[indices, ::2] * -1
        for a, b in self.flip_indices:
            self.targets[indices, a], self.targets[indices, b] = \
            self.targets[indices, b], self.targets[indices, a]
    
    def rotate(self):
        indices = self._random_indices(self.rotate_ratio)
        self.targets = self.targets.reshape(len(self.targets), self.y.shape[1] // 2, 2)
        for i in indices:
            angle = np.random.randint(-10, 10)
            self.inputs[i] = rotate(self.inputs[i], angle)
            angle = angle * np.pi / 180
            C = [[np.cos(angle), -np.sin(angle)],
                 [np.sin(angle), np.cos(angle)]]
            self.targets[i] = np.dot(self.targets[i], C)
        self.targets = self.targets.reshape(-1, self.y.shape[1])
    
    def zoom(self):
        indices = self._random_indices(self.zoom_ratio)
        for i in indices:
            a, b = np.random.randint(0, self.zoom_range, 2)
            self.targets[i] = self.targets[i] * (IMSIZE / 2) + (IMSIZE / 2)
            self.targets[i, ::2] = self.targets[i, ::2] - b
            self.targets[i, 1::2] = self.targets[i, 1::2] - a
            self.targets[i] = 2 * (self.targets[i] - (IMSIZE - self.zoom_range) / 2) / (IMSIZE - self.zoom_range)
            self.inputs[i] = resize(self.inputs[i, a:-self.zoom_range+a, b:-self.zoom_range+b], (IMSIZE, IMSIZE))
            
    def noise(self):
        indices = self._random_indices(self.noise_ratio)
        for i in indices:
            self.inputs[i] = random_noise(self.inputs[i])
    
    def generate(self):
        while True:
            self.X, self.y = shuffle(self.X, self.y)
            start = 0
            stop = self.bs
            for i in range(self.size // self.bs):
                self.inputs = self.X[start:stop].copy()
                self.targets = self.y[start:stop].copy()
                start += self.bs
                stop += self.bs
                self.flip()
                self.rotate()
                self.noise()
                self.zoom()
                yield (self.inputs, self.targets)

In [5]:
def load(n=MAXN, test=False):
    
    cols = np.arange(1, 29)
    go = PATH + 'train/'
    fnames = glob(go + 'images/*')
    X = np.zeros((n, IMSIZE, IMSIZE, 3))
    y = None
    
    if not test:
        y = np.genfromtxt(go + 'gt.csv', delimiter=',', skip_header=1, usecols=cols)
    
    for i, name in enumerate(sorted(fnames)[:n]):
        img = load_img(name, target_size=(IMSIZE, IMSIZE))
        img = img_to_array(img)
        
        if not test:
            h, w = img_to_array(load_img(name)).shape[:2]
            y[i, 0::2] = 2 * (y[i, 0::2] - w / 2) / w
            y[i, 1::2] = 2 * (y[i, 1::2] - h / 2) / h
        
        X[i] = img
    
    X /= 255.
    
    return X, y[:n]

In [6]:
def plot_sample(x, y, axs):
    
    axs.imshow(x)
    sz = x.shape[0]
    axs.scatter(y[0::2] * (sz // 2) + (sz // 2), y[1::2] * (sz // 2) + (sz // 2))

In [7]:
def CNN():
    model = Sequential()
    model.add(Conv2D(32,(3, 3), input_shape = (IMSIZE, IMSIZE, 3)))
    model.add(Activation('relu')) ## 96 - 3 + 2
    model.add(MaxPooling2D(pool_size = (2,2))) ## 96 - (3-1)*2

    model.add(Conv2D(64,(2,2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(Conv2D(128,(2,2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(1000))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(28))
    
    return(model)

In [8]:
def cosine_lr(start, stop, epochs, n):
    epochs /= n
    res = stop + (start - stop) / 2 * (1  + np.cos(np.linspace(0, epochs, epochs) * np.pi / epochs))
    res = np.concatenate([res for i in range(n)])
    return res

In [9]:
X, y = load()

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [11]:
start = 0.001
stop = 0.0001
epochs = 300
learning_rates = cosine_lr(start, stop, epochs, 1)
change_lr = LearningRateScheduler(lambda e: float(learning_rates[e]))
early_stop = EarlyStopping(patience=100)
sgd = SGD(lr=start, momentum=0.9, nesterov=True)
checkpoint = ModelCheckpoint('model.{epoch:04d}-{val_loss:.5f}.hdf5', monitor='val_loss', 
                             period=50, verbose=0)

/home/rickya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
model = CNN()
model.compile(loss='mean_squared_error', optimizer=Adam(lr=start))

In [13]:
bs = 100
gen_train = Generator(X_train, y_train, bs=bs,
                      flip_ratio=0.5,
                      rotate_ratio=0.7,
                      noise_ratio=0,
                      zoom_ratio=0.7,
                      zoom_range=8,
                      flip_indices=flip_indices)

In [14]:
model.fit_generator(gen_train.generate(), verbose=1,
                    steps_per_epoch=X_train.shape[0] // bs,
                    epochs=epochs, validation_data=(X_val, y_val),
                    callbacks=[change_lr, early_stop, checkpoint])

Epoch 1/300


/home/rickya/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


42/51 [=======================>......] - ETA: 10s - loss: 0.0583

MemoryError: 

In [ ]:
model.save('facepoints-model.hdf5')